### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

In [2]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

c:\Users\ChitreshKaushik\Documents\RAG_UseCases\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CC9E363B60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CC9E1E8830>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [5]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})


In [6]:
print(decomposition_question)

Here are some sub-questions to break down the complex query:

1. **How does LangChain utilize memory in its applications?**  (Focuses on LangChain's memory capabilities)
2. **What types of agents are supported by LangChain, and how are they implemented?** (Focuses on LangChain's agent functionality)
3. **How does CrewAI handle memory management within its system?** (Focuses on CrewAI's memory approach)
4. **What are the key differences in agent design and functionality between LangChain and CrewAI?** (Directly compares agent aspects of both) 


These sub-questions allow for more targeted document retrieval by isolating specific aspects of memory and agent usage in both LangChain and CrewAI. 



In [7]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [8]:
sub_qs_text = decomposition_chain.invoke({"question": "How does LangChain use memory and agents compared to CrewAI?"})
sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]

In [9]:
sub_questions

['Here are some sub-questions that break down the complex question:',
 "**How does LangChain manage memory in its applications?** (Focuses on LangChain's specific memory mechanisms)",
 "**What types of agents can be built using LangChain, and how do they leverage memory?** (Explores LangChain's agent capabilities and memory integration)",
 "**How does CrewAI handle memory and agent functionality?** (Focuses on CrewAI's approach to these concepts)",
 '**What are the key differences in memory management and agent design between LangChain and CrewAI?** (Directly compares the two systems)',
 "These sub-questions allow for more focused document retrieval and a clearer understanding of the nuances between LangChain and CrewAI's approaches to memory and agents"]

In [10]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [11]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: Here are some sub-questions that break down the complex query:
A: Please provide the complex query you want to break down. I need the actual question to be able to generate sub-questions.  

For example, you could say:

"Here is the complex query: **How can I use CrewAI to analyze a legal document and summarize the key findings?**  Break this down into sub-questions." 


Then I can help you formulate sub-questions! 😊 


Q: **What memory management techniques does LangChain employ?**  (This focuses on LangChain's specific approach)
A: LangChain uses memory modules like **ConversationBufferMemory** and **ConversationSummaryMemory** to manage memory.  

* **ConversationBufferMemory** allows the LLM to remember previous conversation turns. 
* **ConversationSummaryMemory** summarizes long interactions to keep the context within token limits. 


Let me know if you'd like more details on how these memory modules work! 


Q: **How does LangChain utilize agents for task comp